In [6]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
from path import Path
from sklearn.model_selection import StratifiedKFold
class Config:
    input_path = Path('../input')
    optuna_lgb = False
    n_estimators = 1500
    early_stopping_round = 150
    cv_folds = 5
    random_state = 0
    params = {'objective': 'binary',
              'boosting_type': 'gbdt',
              'learning_rate': 0.01,
              'max_bin': 25,
              'num_leaves': 31,
              'min_child_samples': 1500,
              'colsample_bytree': 0.7,
              'subsample_freq': 1,
              'subsample': 0.7,
              'reg_alpha': 1.0,
              'reg_lambda': 1.0,
              'verbosity': 0,
              'random_state': 0}
    
config = Config()



In [7]:
train = pd.read_csv(config.input_path/'train.csv',index_col='id')
test = pd.read_csv(config.input_path/'test.csv', index_col= 'id')
submission = pd.read_csv(config.input_path/'sample_submission.csv', index_col='id')

calc_features = [feat for feat in train.columns if "_calc" in feat]
cat_features = [feat for feat in train.columns if "_cat" in feat]

In [8]:
target = train['target']
train = train.drop("target", axis  = 'columns')



In [9]:
#removing calc features since they do not accomplish good results anyway
train = train.drop(calc_features, axis = 'columns')
test = test.drop(calc_features, axis = 'columns')

In [10]:
from numba import jit
@jit
def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini
def gini_lgb(y_true, y_pred):
    eval_name = 'normalized_gini_coef'
    eval_result = eval_gini(y_true, y_pred)
    is_higher_better = True
    return eval_name, eval_result, is_higher_better



C:\Users\fuksy\AppData\Local\Temp\ipykernel_19568\1989436427.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [12]:
if config.optuna_lgb:
    def objective(trial):
        params = {
            'learning_rate':trial.suggest_float("learning_rate",0.01,1.0),
            'num_leaves':trial.suggest_int("num_leaves", 3 ,255),
            'min_child_samples':trial.suggest_int("min_child_samples",3,30000),
            'subsample_freq':trial.suggest_int("subsample_freq",0,10),
            'subsample':trial.suggest_float("subsample", 0.1, 1,0),
            'reg_lambda':trial.suggest_loguniform("reg_lambda",1e-9,10.0)
        }

        score = list()
        skf = StratifiedKFold(n_splits = config.cv_folds,
                               shuffle=True,
                               random_state = config.random_state)
        
        for train_idx, valid_idx in skf.split(train,target):
            X_train = train.iloc[train_idx]
            y_train = target.iloc[train_idx]
            X_valid = train.iloc[valid_idx]
            y_valid = target.iloc[valid_idx]

            model = lgb.LGBMClassifier(**params,
                                       n_estimators=1500,
                                       early_stopping = 150,
                                       force_row_wise = True)
            callbacks = [lgb.early_stopping(stopping_rounds=150,
                                            verbose = False)]
            
            model.fit(X_train,y_train,
                      eval_set = [(X_valid,y_valid)],
                      eval_metric=gini_lgb,
                      callbacks=callbacks)
            
            score.append(model.best_score_['valid_0']['normalized_gini_coef'])

        return np.mean(score)
    study = optuna.create_study(direction = 'maximize')
    print("best gini", study.best_value)
    print("best paras", study.best_params)

    params = {
        'objective':'binary',
        ' boosting_type':'gbdt',
        'verbosity':0,
        'random_state':0
    }

    params.update(study.best_params)

else:
    params = config.params


In [13]:
preds = np.zeros(len(test))
oof = np.zeros(len(train))
metric_evaluations = list()
skf = StratifiedKFold(n_splits=config.cv_folds, shuffle=True, random_state=config.random_state)
for idx, (train_idx, valid_idx) in enumerate(skf.split(train, 
                                                       target)):
    print(f"CV fold {idx}")
    X_train, y_train = train.iloc[train_idx], target.iloc[train_idx]
    X_valid, y_valid = train.iloc[valid_idx], target.iloc[valid_idx]
    
    model = lgb.LGBMClassifier(**params,
                               n_estimators=config.n_estimators,
                    early_stopping_round=config.early_stopping_round,
                               force_row_wise=True)
    
    callbacks=[lgb.early_stopping(stopping_rounds=150), 
               lgb.log_evaluation(period=100, show_stdv=False)]
                                                                                           
    model.fit(X_train, y_train, 
              eval_set=[(X_valid, y_valid)], 
              eval_metric=gini_lgb, callbacks=callbacks)
    metric_evaluations.append(
                model.best_score_['valid_0']['normalized_gini_coef'])
    preds += (model.predict_proba(test,  
              num_iteration=model.best_iteration_)[:,1] 
              / skf.n_splits)
    oof[valid_idx] = model.predict_proba(X_valid, 
                    num_iteration=model.best_iteration_)[:,1]



CV fold 0


c:\Users\fuksy\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\fuksy\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153284	valid_0's normalized_gini_coef: 0.272472
[200]	valid_0's binary_logloss: 0.152333	valid_0's normalized_gini_coef: 0.279339
[300]	valid_0's binary_logloss: 0.151907	valid_0's normalized_gini_coef: 0.285607
[400]	valid_0's binary_logloss: 0.151663	valid_0's normalized_gini_coef: 0.28995
[500]	valid_0's binary_logloss: 0.151529	valid_0's normalized_gini_coef: 0.292425
[600]	valid_0's binary_logloss: 0.15146	valid_0's normalized_gini_coef: 0.293545
[700]	valid_0's binary_logloss: 0.151417	valid_0's normalized_gini_coef: 0.294406
[800]	valid_0's binary_logloss: 0.151393	valid_0's normalized_gini_coef: 0.2949
[900]	valid_0's binary_logloss: 0.151392	valid_0's normalized_gini_coef: 0.294881
Early stopping, best iteration is:
[802]	valid_0's binary_logloss: 0.151393	valid_0's normalized_gini_coef: 0.294906
CV fold 1
Training until validation scores don't improve for 150 rounds
[100]	valid_0's

In [15]:
print(f"lightgbm cv normalized gini coeff:\
      {np.mean(metric_evaluations):0.3f},\
      ({np.std(metric_evaluations):0.3f})")

lightgbm cv normalized gini coeff:      0.287,      (0.014)


In [19]:
submission['target'] = preds
submission.to_csv("lgb_submission.csv")

oofs = target.to_frame()
oofs['target'] = oof
oofs.to_csv('lgb_oof.csv')

In [17]:
submission

,target
id,
0,0.0364
1,0.0364
2,0.0364
3,0.0364
4,0.0364
...,...
1488022,0.0364
1488023,0.0364
1488024,0.0364
